In [1]:
import hoomd
import pickle
import gsd.hoomd
import numpy as np
import pandas as pd
import os
import rowan
import warnings
warnings.filterwarnings('ignore')

## Create pps atomistic simulation to collect positions of constituent particles and COM

In [5]:
num_mols = 100
aa_frame = gsd.hoomd.open("assets/pps_aa_N100.gsd")[0]
positions = aa_frame.particles.position
# indices of constituent particles
constituent_indx = [np.arange(7) + (i*7) for i in range(num_mols) ]

In [7]:

com_mass = []
com_positions = []

for indx in constituent_indx:
    total_mass = np.sum(aa_frame.particles.mass[indx])
    com_mass.append(total_mass)
    com_positions.append(
                np.sum(
                    aa_frame.particles.position[indx]
                    * aa_frame.particles.mass[indx, np.newaxis],
                    axis=0,
                )
                / total_mass
            )

In [8]:
rel_const_pos =  positions[constituent_indx[0]] - com_positions[0]
rel_const_pos

array([[ 0.01249123,  0.05568218, -0.64855003],
       [-0.28306198, -0.12721539, -0.47495747],
       [-0.29074383, -0.16056156, -0.08789635],
       [-0.00240135, -0.01087952,  0.12610817],
       [-0.01196671, -0.05335712,  0.621377  ],
       [ 0.293334  ,  0.17228031, -0.048069  ],
       [ 0.30057716,  0.20537663, -0.43515968]], dtype=float32)

In [14]:
from flowermd.library import PPS, OPLS_AA_PPS
from flowermd.base import Pack
pps = PPS(num_mols=2, lengths=1)
system = Pack(molecules=pps, density=0.2)
system.apply_forcefield(r_cut=2.5, force_field=OPLS_AA_PPS(), auto_scale=True, remove_charges=True, scale_charges=True, remove_hydrogens=True)
pps_ff = system.hoomd_forcefield

No charged group detected, skipping electrostatics.


/home/marjan/anaconda3/envs/anisoML/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value
/home/marjan/anaconda3/envs/anisoML/lib/python3.11/site-packages/gmso/core/topology.py:818: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_scales[index][scaling_interaction_idxes[interaction]] = value


In [15]:
pps_ff

## Create rigid body simulation

In [20]:
def create_rigid_simulation(kT):
    rigid_simulation = hoomd.Simulation(device=hoomd.device.auto_select(), seed=1)
    rigid_simulation.create_state_from_gsd(filename='assets/pps_rigid_lattice_100.gsd')
    const_particle_types = list(np.asarray(aa_frame.particles.types)[list(aa_frame.particles.typeid)][constituent_indx[0]])

    rigid = hoomd.md.constrain.Rigid()
    rigid.body['rigid'] = {
        "constituent_types":const_particle_types,
        "positions": rel_const_pos,
        "orientations": [(1.0, 0.0, 0.0, 0.0)]* len(rel_const_pos),
        }
    integrator = hoomd.md.Integrator(dt=0.005, integrate_rotational_dof=True)
    rigid_simulation.operations.integrator = integrator
    integrator.rigid = rigid
    rigid_centers_and_free = hoomd.filter.Rigid(("center", "free"))
    nvt = hoomd.md.methods.ConstantVolume(
        filter=rigid_centers_and_free,
        thermostat=hoomd.md.methods.thermostats.Bussi(kT=kT))
    integrator.methods.append(nvt)
    
    cell = hoomd.md.nlist.Cell(buffer=0, exclusions=['body'])
    
    lj = hoomd.md.pair.LJ(nlist=cell)
    
    # use aa pps simulation to define lj and special lj forces between constituent particles
    for k, v in dict(pps_ff[0].params).items():
        lj.params[k] = v
        lj.r_cut[k] = 2.5
    
    lj.params[('rigid', ['rigid', 'ca', 'sh'])]= dict(epsilon=0, sigma=0)
    lj.r_cut[('rigid', ['rigid', 'ca', 'sh'])] = 0

    integrator.forces.append(lj)
    rigid_simulation.state.thermalize_particle_momenta(filter=rigid_centers_and_free,
                                             kT=kT)
    
    rigid_simulation.run(0)

    log_quantities = [
                        "kinetic_temperature",
                        "potential_energy",
                        "kinetic_energy",
                        "volume",
                        "pressure",
                        "pressure_tensor",
                    ]
    logger = hoomd.logging.Logger(categories=["scalar", "string", "particle"])
    logger.add(rigid_simulation, quantities=["timestep", "tps"])
    thermo_props = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
    rigid_simulation.operations.computes.append(thermo_props)
    logger.add(thermo_props, quantities=log_quantities)
    
    # for f in integrator.forces:
    #     logger.add(f, quantities=["energy", "forces", "energies"])

    logger.add(rigid_simulation.operations.integrator.rigid, quantities=["torques", "forces", "energies"])
    
    gsd_writer = hoomd.write.GSD(
        filename="logs/pps_100/trajectory_{}.gsd".format(kT),
        trigger=hoomd.trigger.Periodic(int(100)),
        mode="wb",
        logger=logger,
        filter=hoomd.filter.All(),
        dynamic=["momentum", "property"]
        )
    
    rigid_simulation.operations.writers.append(gsd_writer)
    return rigid_simulation

    


In [30]:
kT_list = [1., 1.5, 2., 3., 6.0]
for kT in kT_list:
    rigid_simulation = create_rigid_simulation(kT)
    rigid_simulation.run(2e5)
    rigid_simulation.operations.writers[0].flush()

In [28]:
kT

1.0

## filter data from r > 3.5

In [31]:
positions = []
orientations = []
forces = []
torques = []
energies = []
for kT in kT_list:
    traj = gsd.hoomd.open("logs/pps_100/trajectory_{}.gsd".format(kT))
    for frame in traj[100:]:
        positions.append(frame.particles.position[:num_mols])
        orientations.append(frame.particles.orientation[:num_mols])
        forces.append(frame.log['particles/md/constrain/Rigid/forces'][:num_mols])
        torques.append(frame.log['particles/md/constrain/Rigid/torques'][:num_mols])
        energies.append(frame.log['particles/md/constrain/Rigid/energies'][:num_mols])
        

In [39]:
len(positions)

9500

## create the dataset

In [54]:

particle_pos = []
neighbor_pos = []
particle_orient_q = []
neighbor_orient_q = []
particle_orient_R = []
neighbor_orient_R = []
particle_force = []
particle_torque = []
particle_energy = []
counter = 0
for pos, orient, force, torque, energy in zip(positions, orientations, forces,torques, energies):
    for idx in range(num_mols):
        particle_pos.append(pos[idx])
        mask = np.ones(num_mols, dtype=bool)
        mask[idx] = False
        neighbor_pos.append(pos[mask])
    
        particle_orient_q.append(orient[idx])
        neighbor_orient_q.append(orient[mask])
    
        particle_orient_R.append(rowan.to_matrix(orient[idx]))
        neighbor_orient_R.append(rowan.to_matrix(orient[mask]))
    
        particle_force.append(force[idx])
        particle_torque.append(torque[idx])
        particle_energy.append(energy[idx])
    counter += 1
    if counter % 50 == 0:
        print(counter)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500


In [55]:
len(neighbor_orient_q)

950000

In [56]:

columns = [
    "particle_pos",
    "neighbor_pos",
    "particle_orient_q",
    "neighbor_orient_q", 
    "particle_orient_R",
    "neighbor_orient_R",
    "force",
    "torque",
    "energy"
]

new_traj_df = pd.DataFrame(columns=columns)
new_traj_df["particle_pos"] = particle_pos
new_traj_df["neighbor_pos"] = neighbor_pos
new_traj_df["particle_orient_q"] = particle_orient_q
new_traj_df["neighbor_orient_q"] = neighbor_orient_q
new_traj_df["particle_orient_R"] = particle_orient_R
new_traj_df["neighbor_orient_R"] = neighbor_orient_R
new_traj_df["force"] = particle_force
new_traj_df["torque"] = particle_torque
new_traj_df["energy"] = particle_energy

In [57]:
new_traj_df.head()

,particle_pos,neighbor_pos,particle_orient_q,neighbor_orient_q,particle_orient_R,neighbor_orient_R,force,torque,energy
0,"[-8.966934, -3.0756888, 7.652912]","[[-1.4110719, -8.304523, 1.9127961], [12.98338...","[-0.3504857, 0.12579411, -0.9153582, 0.15315022]","[[-0.21375161, -0.5644795, -0.793161, -0.08104...","[[-0.7226712538404021, -0.12293942272663116, 0...","[[[-0.2713463306427002, 0.8607977628707886, 0....","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",0.000000
1,"[-1.4110719, -8.304523, 1.9127961]","[[-8.966934, -3.0756888, 7.652912], [12.983382...","[-0.21375161, -0.5644795, -0.793161, -0.08104825]","[[-0.3504857, 0.12579411, -0.9153582, 0.153150...","[[-0.2713462872115544, 0.8607977628707886, 0.4...","[[[-0.722671389579773, -0.12293942272663116, 0...","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",0.000000
2,"[12.983382, 14.533489, 4.208566]","[[-8.966934, -3.0756888, 7.652912], [-1.411071...","[-0.18609919, 0.85284764, 0.41232643, -0.26077...","[[-0.3504857, 0.12579411, -0.9153582, 0.153150...","[[0.523963987827301, 0.6062422394752502, -0.59...","[[[-0.722671389579773, -0.12293942272663116, 0...","[1.475818032595689, -18.28394715178201, -5.089...","[0.6710396531517533, 1.7926970523506705, -9.29...",-2.808060
3,"[15.093127, 12.034294, -2.914589]","[[-8.966934, -3.0756888, 7.652912], [-1.411071...","[-0.17227277, 0.16530788, -0.78147036, -0.5764...","[[-0.3504857, 0.12579411, -0.9153582, 0.153150...","[[-0.8859907388687134, -0.4569811224937439, 0....","[[[-0.722671389579773, -0.12293942272663116, 0...","[-2.5761640634928233, 1.2957694232617478, -0.1...","[0.04918616221577604, -2.7167493599761223, 2.4...",-4.638836
4,"[15.467617, -4.7798147, -15.156477]","[[-8.966934, -3.0756888, 7.652912], [-1.411071...","[0.048202466, 0.5801122, 0.8115599, -0.050168067]","[[-0.3504857, 0.12579411, -0.9153582, 0.153150...","[[-0.32229264511422184, 0.9464280605316162, 0....","[[[-0.722671389579773, -0.12293942272663116, 0...","[12.290791460906057, -4.169066822921853, 1.750...","[0.5243822386990278, 1.8943394408648202, -0.21...",-2.205088


In [58]:
# shuffle dataset 
new_traj_df = new_traj_df.sample(frac=1).reset_index(drop=True)

In [59]:
new_traj_df.head()

,particle_pos,neighbor_pos,particle_orient_q,neighbor_orient_q,particle_orient_R,neighbor_orient_R,force,torque,energy
0,"[-4.683796, 10.292817, -13.21526]","[[9.460879, -13.591212, 2.2277095], [2.7115068...","[0.6412764, 0.60312, 0.277415, -0.3847748]","[[-0.5299731, -0.072839506, -0.3492191, -0.769...","[[0.5499785244464874, 0.8281230926513672, -0.1...","[[[-0.42764580249786377, -0.7645747065544128, ...","[0.32490627026652713, -0.856436283102786, -0.5...","[0.2739095265325195, -0.022535769544965882, 0....",-0.180284
1,"[14.868779, 15.727235, -1.3359331]","[[-5.8637366, 5.255882, 6.850473], [-9.166343,...","[-0.6198705, -0.7215969, 0.30648297, 0.033566188]","[[-0.63796794, -0.49930328, -0.0753164, 0.5813...","[[0.8098829871346347, -0.40070095658302307, -0...","[[[0.31261366605758667, 0.8170347213745117, -0...","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",0.000000
2,"[-14.908929, 1.1536107, 6.6091895]","[[14.56052, -9.22868, 3.7860055], [8.985262, -...","[-0.89804035, -0.28828517, -0.25767124, 0.2098...","[[0.1522114, 0.95940614, -0.23476051, -0.03548...","[[0.7791696059208874, 0.5254033803939819, 0.34...","[[[0.887256920337677, -0.4396595358848572, -0....","[-0.20880214212862006, 1.4457680488900875, 0.8...","[0.38175755167368675, 0.12528367897783058, -0....",-0.284747
3,"[-9.929115, 12.854807, -9.628236]","[[11.403184, -6.777678, -14.915941], [7.60905,...","[-0.17527968, 0.098626286, -0.8505628, -0.4858...","[[-0.38858575, 0.5649842, 0.7277001, 0.0157011...","[[-0.9190998077392578, -0.3381103277206421, 0....","[[[-0.059587955474853516, 0.8344805836677551, ...","[-0.004589357756950026, -0.02539214991023336, ...","[0.007710328699464657, -0.0013822168095516578,...",-0.005127
4,"[-9.651248, -3.7597475, -15.1244545]","[[7.3380804, 7.8279324, 11.853653], [-12.77024...","[-0.4053136, 0.066194795, -0.85599566, 0.31402...","[[0.64522505, -0.47349116, -0.40121615, -0.445...","[[-0.6626782216237916, 0.14123079180717468, 0....","[[[0.28101855516433716, 0.9549068212509155, -0...","[-3.231219959399344, -1.273251866523756, -3.71...","[-0.45380868397604446, -0.6266873970117057, 0....",-1.346116


In [60]:
target_datapath="/home/marjan/Documents/code-base/ml_datasets/pps_100"

new_traj_df.to_pickle(os.path.join(target_datapath, "raw.pkl"))
test_frac = 0.1
val_frac = 0.1

dataset_len = new_traj_df.shape[0]
test_len = int(dataset_len * test_frac)
val_len = int(dataset_len * val_frac)

test_df = new_traj_df.iloc[:test_len]

val_df = new_traj_df.iloc[test_len: test_len + val_len]

train_df = new_traj_df.iloc[test_len + val_len:]


In [61]:
train_df.to_pickle(os.path.join(target_datapath, 'train.pkl'))
val_df.to_pickle(os.path.join(target_datapath, 'val.pkl'))
test_df.to_pickle(os.path.join(target_datapath, 'test.pkl'))

In [62]:
print(test_df.shape)

(95000, 9)


In [63]:
print(val_df.shape)

(95000, 9)


In [64]:
print(train_df.shape)

(760000, 9)
